In [ ]:
import torch 
from torch.linalg import vector_norm
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import dijkstra

from distances import *
from methods import * 
from optimizers import * 


latent_dim = 2
lr = 0.01
num_epochs = 100000
normalize = True

geodesic = True
min_dist = 1.

data = torch.randn(10, 10)
data_dist_matrix = dist_matrix(data, Euclidean)

#IsoMap-style geodesic distance for data
if geodesic:
#     truncated_matrix = torch.where(data_dist_matrix < min_dist, data_dist_matrix, torch.inf)
#     data_dist_matrix = dijkstra(truncated_matrix.detach().cpu().numpy())
#     data_dist_matrix = torch.FloatTensor(data_dist_matrix)
#     data_dist_matrix = torch.where(data_dist_matrix == torch.inf, 1000 * torch.ones_like(data_dist_matrix), data_dist_matrix)
    
    data_nn_matrix = kneighbors_graph(data, 3, mode='distance', include_self=False)
    data_nn_matrix = data_nn_matrix.toarray()
    data_dist_matrix = dijkstra(data_nn_matrix)
    data_dist_matrix = torch.FloatTensor(data_dist_matrix)
    data_dist_matrix = torch.where(data_dist_matrix == torch.inf, 1000 * torch.ones_like(data_dist_matrix), data_dist_matrix)
       
    

# torch.manual_seed(42)       
# torch.cuda.empty_cache()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print('Using device: ', device)

    


#model = MDS(data.shape[0], latent_dim, Poincare)
model = Isomap(data.shape[0], latent_dim, Poincare)

#optimizer = StandardOptim(model, lr=lr)
optimizer = PoincareOptim(model, lr=lr)

if __name__ == "__main__":
    for i in range(num_epochs):
        if normalize:
            model.normalize()

        # print('norms', vector_norm(model.embeddings, dim=-1).mean().item(), vector_norm(model.embeddings, dim=-1).max().item())

        optimizer.zero_grad()
        loss = model.loss_fun(data_dist_matrix)
        loss.backward()
        # print('grads', vector_norm(model.embeddings.grad, dim=-1).mean().item())
        optimizer.step()
        

        if i % 10 == 0:
            print(f'Epoch {i}, loss: {loss:.3f}')



In [8]:
data.shape[0]

10